This notebook pre-processes designs from the [photonics-opt-testbed](https://github.com/NanoComp/photonics-opt-testbed/tree/main/RGB_metalens) repo, e.g. by trimming, adjusting resolution, and padding to achieve the expected shape.

In [ ]:
import glob

import matplotlib.pyplot as plt
import numpy as onp

from invrs_gym.utils import transforms
from invrs_gym.challenges.metalens import challenge


def process_reference_design(fname):
    original_design = onp.genfromtxt(fname, delimiter=",")
    design = original_design.copy()

    # The `Ex` designs have solid on the substrate side, and void on the ambient side.
    # The `Ey` designs are completely surrounded by void.
    design_type = fname.split("/")[-2]
    assert design_type in ("Ex", "Ez")
    pad_value_cap = 0
    pad_value_base = 0 if design_type == "Ez" else 1

    lo = 0
    for i in range(design.shape[1]):
        if not onp.all(design[:, i] == pad_value_base):
            lo = i
            break

    hi = design.shape[1]
    for i in range(design.shape[1], -1, -1):
        if not onp.all(design[:, i - 1] == pad_value_cap):
            hi = i
            break

    assert lo == 0 or onp.all(design[:, lo - 1] == pad_value_base)
    assert hi == design.shape[1] - 1 or onp.all(design[:, hi + 1] == pad_value_cap)
    design = design[:, lo:hi]

    # Special treatment for the `Ex/Rasmus70nm.csv` design, which includes a void
    # column of pixels at the base (adjacent to the substrate). We assume this is
    # an error and perform additional cropping, so that the lens is connected to the
    # substrate and not floating above it.
    if fname.endswith("Ex/Rasmus70nm.csv"):
        if original_design.shape == (1022, 122):
            # Ensure additional cropping is only done for original unmodified design.
            assert onp.all(design[:, 0] == 0)
            assert onp.all(design[10:-10, 1:10] == 1)
            design = design[:, 10:]
            design = onp.pad(design, ((1, 1), (0, 0)), mode="edge")

    # Flip the design, as is the convention for the invrs-gym challenge.
    design = design[:, ::-1]

    # Decrease the resolution of the "Rasmus" designs.
    if "Rasmus" in fname:
        design = onp.pad(design, ((0, 0), (design.shape[1] % 2, 0)))
        assert design.shape[0] % 2 == design.shape[1] % 2 == 0
        shape = (design.shape[0] // 2, design.shape[1] // 2)
        design = onp.array(transforms.box_downsample(design, shape))
        assert design.shape[0] % 2 == 0

    # Add a row of entirely zeros to the top, and entirely ones to the bottom.
    design = onp.pad(design, ((0, 0), (1, 0)), constant_values=pad_value_cap)
    design = onp.pad(design, ((0, 0), (0, 1)), constant_values=pad_value_base)

    # Pad the design to the full shape expected by the challenge.
    grid_shape = challenge.METALENS_SPEC.grid_shape
    pad_width = (grid_shape[0] - design.shape[0]) // 2
    design = onp.pad(design, ((pad_width, pad_width), (0, 0)), mode="edge")

    output_fname = fname.split("/")[-1]
    onp.savetxt(output_fname, design, fmt="%.2f", delimiter=",")

    return design, original_design


fnames = glob.glob("raw/Ex/*.csv")
plt.figure(figsize=(14, 1.6 * len(fnames)))
for i, fname in enumerate(fnames):
    design, original_design = process_reference_design(fname)
    ax = plt.subplot(len(fnames), 2, 2 * i + 1)
    im = ax.imshow(original_design.T, cmap="gray")
    ax.axis(False)

    ax = plt.subplot(len(fnames), 2, 2 * i + 2)
    im = ax.imshow(design.T, cmap="gray")
    ax.axis(False)